In [2]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

In [3]:
llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [4]:
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b

In [5]:
tools = [add, multiply, divide]
llm_with_tools = llm.bind_tools(tools)

sys_msg = SystemMessage(
    content="Use either tools to perform arithmetic on a set of inputs or just give me answer with your knowledge")


In [6]:
def assistant(state: MessagesState):
   return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

In [7]:
# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "assistant")
builder.add_edge("assistant", END)

In [8]:
react_graph = builder.compile()

In [9]:
messages = [HumanMessage(content="Add 3 and 4")]
messages = react_graph.invoke({"messages": messages})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Add 3 and 4
================================== Ai Message ==================================
Tool Calls:
  add (63b92759-2913-4716-b6ab-19bf1ba56bb6)
 Call ID: 63b92759-2913-4716-b6ab-19bf1ba56bb6
  Args:
    a: 3
    b: 4
================================= Tool Message =================================
Name: add

7
================================== Ai Message ==================================

The sum of 3 and 4 is 7.


In [10]:
messages = [HumanMessage(content="Multiply that by 10")]
messages = react_graph.invoke({"messages": messages})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Multiply that by 10
================================== Ai Message ==================================

[{'type': 'text', 'text': 'What number would you like to multiply by 10?', 'extras': {'signature': 'CqsHAdHtim8Z8aGyNUh8CZpn+nNnZj+HWSrcrgHCiAG5UXRlWqdQKxpa9Gh1935IVEhAIMiuc4luCeBPJET88QJBhB4umtBRzHTAqZwQSn331KTGmKAk2yHFqLggdj+pFVnkZq8jhuwgoDjEbvBian8Nd6g/TFc/XBaXAnoAA+ywb9C9jZIvBJdjWzmbMgQGKIpTamJNAlFxeFql2CZoVAp0Uu2h+ViPflrdTz3pz0XWaGnLtdhWbkeKYNBVkphqFDIbP5dtuKIBJP/eqzaofPZnHRv7HuGaRzHAOtYzUhKqJLW/9iTfStzpUzbHS9iQ5p6m1aYgz66O8VsTbCmmdrChqQNx9uJFv7yVEtWvXFUikGhU92yk+kJsp0gAhxOFJybvg+36pNy2iJM/TKg6C3w7FB796XyL4KUNE+E3mFHvxlNdZQd1upPCW5ehm4JnEZofQJIl8Oiiub5QV8TwAEOxNNgv/cdIxisCEk1swULhXJbDIyxzk21qGM0gx0H//uPiLakPihPhmu3ylUCBY2ybMnqNikvTJB7k/d8GrdlwS+u4yrIJIGvHQx7NT8O9GuQVUDBDGhxW5kJiL9B5n72Uz0Uf/RHaPI6giptN8vcpDL4x/87wfrIIjoz1QDxx82GJk/qHPipbk4LoR9qVrA1X3wPUuQoIqvYRoH6aU4naC+yMKlNqvomxsDTg4Z4uOLqcyY1MNa4U